In [23]:
##############comments#############
#22/11/2023 11:47pm - saves grid images in a folder in reverse order 


In [68]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os
from matplotlib import pyplot as plt
import sys
import csv
import shutil
from tqdm.notebook import tqdm
import time



In [72]:

# Step 1: Get all .npy file paths in a folder
folder_path_images_axial = 'MRNet-v1.0/valid/axial'  # Update this with the path to your folder containing .npy files
file_names = sorted(os.listdir(folder_path_images_axial))
file_paths_axial = [os.path.join(folder_path_images_axial, file_name) for file_name in file_names if file_name.endswith('.npy')]

folder_path_images_coronal = 'MRNet-v1.0/valid/coronal'  # Update this with the path to your folder containing .npy files
file_names = sorted(os.listdir(folder_path_images_coronal))
file_paths_coronal = [os.path.join(folder_path_images_coronal, file_name) for file_name in file_names if file_name.endswith('.npy')]

folder_path_images_sagittal = 'MRNet-v1.0/valid/sagittal'  # Update this with the path to your folder containing .npy files
file_names = sorted(os.listdir(folder_path_images_sagittal))
file_paths_sagittal = [os.path.join(folder_path_images_sagittal, file_name) for file_name in file_names if file_name.endswith('.npy')]


#Step 2 get current path and create new folder to save data in current path
path_axial = os.path.join(os.getcwd(), folder_path_images_axial + '4x4_grid_images_cropped')
path_coronal = os.path.join(os.getcwd(), folder_path_images_coronal + '4x4_grid_images_cropped')
path_sagittal = os.path.join(os.getcwd(), folder_path_images_sagittal + '4x4_grid_images_cropped')


if os.path.isdir(path_axial):
    print("Folder: {", os.path.basename(path_axial), "} already exists! Create a new folder name or images will be put into pre-existing folder!")
else:
    os.makedirs(path_axial)
    print("Axial data will be saved in:")
    print(path_axial)
    
if os.path.isdir(path_coronal):
    print("Folder: {", os.path.basename(path_coronal), "} already exists! Create a new folder name or images will be put into pre-existing folder!")
else:
    os.makedirs(path_coronal)
    print("Coronal data will be saved in:")
    print(path_coronal)
    
if os.path.isdir(path_sagittal):
    print("Folder: {", os.path.basename(path_sagittal), "} already exists! Create a new folder name or images will be put into pre-existing folder!")
else:
    os.makedirs(path_sagittal)
    print("Sagittal data will be saved in:")
    print(path_sagittal)
    

Folder: { axial4x4_grid_images_cropped } already exists! Create a new folder name or images will be put into pre-existing folder!
Folder: { coronal4x4_grid_images_cropped } already exists! Create a new folder name or images will be put into pre-existing folder!
Sagittal data will be saved in:
/Users/danielsloseris/Desktop/CMPT 340/Project/ImageClassification/MRNet-v1.0/valid/sagittal4x4_grid_images_cropped


In [77]:


#Step 3: iterate over all .npy volumes in folder and generate grids of the slices laid out in a square
pbar = tqdm(total=len(file_paths_axial), desc = "Creating Grid Images", colour='#669bbc')
count = 0

for file_path_axial, file_path_coronal, file_path_sagittal in zip(file_paths_axial, file_paths_coronal, file_paths_sagittal):
# for file_path in file_paths:
    
    #load volumes
    vol_axial = np.load(file_path_axial)
    vol_coronal = np.load(file_path_coronal)
    vol_sagittal = np.load(file_path_sagittal)
    
    nSlices_axial = np.size(vol_axial,0)#num of slices in image
    nSlices_coronal = np.size(vol_coronal,0)
    nSlices_sagittal = np.size(vol_sagittal,0)
    # print("num Slices:",nSlices)
    
    volName = os.path.basename(file_path_axial) #name will be same for all axes (just finding the label 0000, 0001 etc.)
    end = volName.find('.')
    volName = volName[:end]  

    # print(f"File Name: {os.path.basename(file_path)}")
         
    #add all slices into a single array "images", a PIL, Image object
    slices_axial= []
    slices_coronal = []
    slices_sagittal = []
    for i in range(nSlices_axial):
        image = Image.fromarray(vol_axial[i,:,:])
        # image = image.resize((128,128), 1)# can resize images to make smaller and reduce compute
        w, h = image.size
        image = image.crop((w//4,h//4 , w-w//4, h-h//4))
        slices_axial.append(image)
        
    for i in range(nSlices_coronal):
        image = Image.fromarray(vol_coronal[i,:,:])
        # image = image.resize((128,128), 1)# can resize images to make smaller and reduce compute
        w, h = image.size
        image = image.crop((w//4,h//4 , w-w//4, h-h//4))
        slices_coronal.append(image)
        
    for i in range(nSlices_sagittal):
        image = Image.fromarray(vol_sagittal[i,:,:])
        # image = image.resize((128,128), 1)# can resize images to make smaller and reduce compute
        w, h = image.size
        image = image.crop((w//4,h//4 , w-w//4, h-h//4))
        slices_sagittal.append(image)
        
      #if want to create largest possible grids  
#     gridEdgeSize = int(np.sqrt(nSlices)) #will round down to smallest smaller edge size for a squrae grid
#     nSlicesInGrid = gridEdgeSize**2 #number of slices to include in grid to ensure square
    
#     nSlices_Excluded = nSlices-nSlicesInGrid #number of slices excluded to ensure perfect square (rounding to nearest square)
    
    
    
    #if want to normalize to nxn grids
    n=4
    nSlices_Excluded = nSlices_axial - n**2
    
    #divide the number of exluded slices into 2 to remove from beginning and end, 
    #to hopefully capture more information in middle slices
    nSlices_Excluded_first_half = nSlices_Excluded//2
    nSlices_Excluded_second_half = nSlices_Excluded-nSlices_Excluded_first_half
    start = nSlices_Excluded_first_half
    end = nSlices_axial-nSlices_Excluded_second_half
    grid_axial = pil_grid(slices_axial[start:end],n)

    nSlices_Excluded = nSlices_coronal - n**2
    
    nSlices_Excluded_first_half = nSlices_Excluded//2
    nSlices_Excluded_second_half = nSlices_Excluded-nSlices_Excluded_first_half
    start = nSlices_Excluded_first_half
    end = nSlices_coronal-nSlices_Excluded_second_half
    grid_coronal = pil_grid(slices_coronal[start:end],n)
    

    nSlices_Excluded = nSlices_sagittal - n**2
    
    nSlices_Excluded_first_half = nSlices_Excluded//2
    nSlices_Excluded_second_half = nSlices_Excluded-nSlices_Excluded_first_half
    start = nSlices_Excluded_first_half
    end = nSlices_sagittal-nSlices_Excluded_second_half
    grid_sagittal = pil_grid(slices_sagittal[start:end],n)
    
    
    
    
    # print("File Name: {} | total slices: {}, slices included: {}".format(os.path.basename(file_path),nSlices,len(slices[start:end]) ))
    
    #(use this for rounding to smallest square)
    # grid = pil_grid(slices[start:end],gridEdgeSize) #removes half of excluded slices from beginning and half from end 
    
     #given a defined size of grid we want to make (nxn) use this
    
    #depending on the values at position intVolName on the csvs (info stored in dataArray) place image in correct folder
    grid_axial.save(os.path.join(path_axial, 'axial_grid{}.jpeg'.format(volName)))
    grid_coronal.save(os.path.join(path_coronal, 'coronal_grid{}.jpeg'.format(volName)))
    grid_sagittal.save(os.path.join(path_sagittal, 'sagittal_grid{}.jpeg'.format(volName)))
    count = count+1
    pbar.update(1)
    
pbar.close()
print("\n===================DONE=======================")
print("Num grid Images created:",count)





Creating Grid Images:   0%|          | 0/120 [00:00<?, ?it/s]


===================DONE=======================
Num grid Images created: 120


In [37]:
minSlices = np.size(np.load(file_paths[0]),0)
for file_path in file_paths:
    
    #load volume
    vol = np.load(file_path)
    nSlices = np.size(vol,0)
    if nSlices <minSlices:
        minSlices = nSlices
        
print("min slices", minSlices)

min slices 21


<font size=5> combine 3 grids into one image

In [110]:
folder_path = 'MRNet-v1.0/valid_4x4_cropped'
data_dir1 = os.path.join(folder_path, 'axial4x4_grid_images_cropped_1_fifth_154x154') #axial
data_dir2 = os.path.join(folder_path, 'coronal4x4_grid_images_cropped_1_fifth_154x154') #coronal
data_dir3 = os.path.join(folder_path, 'sagittal4x4_grid_images_cropped_1_fifth_154x154') #sagittal


#Step 2 get current path and create new folder to save data in current path
path = os.path.join(os.getcwd(), folder_path, 'rgb_stacks_4x4_cropped')


if os.path.isdir(path):
    print("Folder: {", os.path.basename(path), "} already exists! Create a new folder name or images will be put into pre-existing folder!")
else:
    os.makedirs(path)
    print("Data will be saved in:")
    print(path)
    
    
file_names1 = sorted(os.listdir(data_dir1))
file_paths_axial = [os.path.join(data_dir1, file_name) for file_name in file_names1 if file_name.endswith('.jpeg')]

file_names2 = sorted(os.listdir(data_dir2))
file_paths_coronal = [os.path.join(data_dir2, file_name) for file_name in file_names2 if file_name.endswith('.jpeg')]

file_names3 = sorted(os.listdir(data_dir3))
file_paths_sagittal = [os.path.join(data_dir3, file_name) for file_name in file_names3 if file_name.endswith('.jpeg')]

Data will be saved in:
/Users/danielsloseris/Desktop/CMPT 340/Project/ImageClassification/MRNet-v1.0/valid_4x4_cropped/rgb_stacks_4x4_cropped


<font size=5> Testing on a single set of images (3 images side by side)

In [108]:


im1 = Image.open(file_paths_axial[0])
im2 = Image.open(file_paths_coronal[0])
im3 = Image.open(file_paths_sagittal[0])
image_size = im1.size
print(image_size)


volName = os.path.basename(file_paths_axial[1]) #can be any, just want the first 4 digits
end = volName.find('.')
start = len(volName)-4
print("volname pre:", volName, "volName len", len(volName[:end]))
volName = volName[len(volName[:end])-4:end]
print("volNAME:",volName)
    
    
grid = pil_grid([im1,im2,im3])
grid.show()



(512, 512)
volname pre: axial_grid0001.jpeg volName len 14
volNAME: 0001


<font size=5> Combine all images in folder into single images (made of 3 grids)

In [67]:

count = 0
pbar = tqdm(total=len(file_paths_axial), desc = "Combining Grid Images", colour='#669bbc')
for file_path_axial, file_path_coronal, file_path_sagittal in zip(file_paths_axial, file_paths_coronal, file_paths_sagittal):

    axial_img = Image.open(file_path_axial)
    coronal_img = Image.open(file_path_coronal)
    sagittal_img = Image.open(file_path_sagittal)
    
    imName = os.path.basename(file_path_axial) #can be any, just want the first 4 digits
    end = imName.find('.')
    # start = len(volName[:end])-4
    imName = imName[len(imName[:end])-4:end]
    # print("volNAME",imName)
    
    grid = pil_grid([axial_img,coronal_img,sagittal_img])
    
    grid.save(os.path.join(path, 'tripleGrid_{}.jpeg'.format(imName)))
    pbar.update(1)
    count = count+1
   

pbar.close()
print("\n===================DONE=======================")
print("Num triple grid Images created:",count)

Combining Grid Images:   0%|          | 0/120 [00:00<?, ?it/s]


===================DONE=======================
Num triple grid Images created: 120


<font size=5> 3 images (axial, coronal, sagittal) as channels of RGB image

In [103]:
im1 = Image.open(file_paths_axial[0])
im2 = Image.open(file_paths_coronal[0])
im3 = Image.open(file_paths_sagittal[0])

#convert rgb images into single channel images type greyscale
im1_s = im1.convert('L') #s for single channel
im2_s = im2.convert('L')
im3_s = im3.convert('L')


rgb = np.dstack((np.array(im1_s),np.array(im2_s),np.array(im3_s)))


rgb_im = Image.fromarray(rgb)
rgb_im.show()

volname pre: axial_grid1131.jpeg volName len 14
volNAME: 1131


<font size=5> Combine all images in folder into single images (made of rbg stacked images)

In [111]:

count = 0
pbar = tqdm(total=len(file_paths_axial), desc = "Combining Grid Images", colour='#669bbc')
for file_path_axial, file_path_coronal, file_path_sagittal in zip(file_paths_axial, file_paths_coronal, file_paths_sagittal):

    axial_img = Image.open(file_path_axial)
    coronal_img = Image.open(file_path_coronal)
    sagittal_img = Image.open(file_path_sagittal)
    
    im1_s = axial_img.convert('L') #s for single channel
    im2_s = coronal_img.convert('L')
    im3_s = sagittal_img.convert('L')
    rgb = np.dstack((np.array(im1_s),np.array(im2_s),np.array(im3_s)))
    rgb_im = Image.fromarray(rgb)
    
    imName = os.path.basename(file_path_axial) #can be any, just want the first 4 digits
    end = imName.find('.')
    # start = len(volName[:end])-4
    imName = imName[len(imName[:end])-4:end]
    # print("volNAME",imName)
    
    # grid = pil_grid([axial_img,coronal_img,sagittal_img])
    
    rgb_im.save(os.path.join(path, 'rgbAxes_{}.jpeg'.format(imName)))
    pbar.update(1)
    count = count+1
   

pbar.close()
print("\n===================DONE=======================")
print("Num rgb triple channel Images created:",count)

Combining Grid Images:   0%|          | 0/120 [00:00<?, ?it/s]


===================DONE=======================
Num rgb triple channel Images created: 120
